# **Máquinas suportadas por Vetor (SVM)**

As máquinas suportadas por vetor são métodos de aprendizado supervisionados que podem ser lineares ou não-lineares, dependendo das hipóteses utilizadas na sua construção.

São algoritmos de alta capacidade de aprendizado e de generalização, mesmo que a hipótese que os baseia seja muito simples. Isso se deve à sua elaborada construção matemática, que está fora do escopo dessa aula.

## **Classificadores de Margem**

Considere o seguinte dataset supervisionado de um problema de classificação binário:

<img src=https://s3-sa-east-1.amazonaws.com/lcpi/d8d83e07-66d8-47c6-a5e4-4a3e232481e2.PNG width=400>

É visível que os dados são linearmente separáveis. De fato, existem infinitas retas possíveis que separam perfeitamente as duas classes.

Apesar disso (erro de treino é nulo!), podemos nos perguntar: qual deles tem potencial de apresentar **melhor generalização?**

Dependendo da inclinação da reta que escolhemos para separar as classes, o erro de generalização pode ser diferente. Precisamos ter um método que determine a melhor reta que garanta error de treinamento baixo, porém que isso não prejudique a classificação dos pontos futuros.

É aqui que entra o conceito de **margem**:

> Chamamos de **margem** a **menor distância** entre os pontos de treino e a fronteira de decisão

Intituivamente, um classificador de margem máxima terá mais chance de generalizar bem para futuros pontos, uma vez que apresenta uma "banda morta" que garante a separação máxima entre as classes.

> Chamamos de **classificador de margem máxima** um classificador **linear** que é construído de modo que a margem seja maximizada.


### **Classificador de Margem Suave**

Um classificador de margem máxima, apesar de ajudar a melhorar a generalização da separação, ele é limitado a dados linearmente separáveis. Além disso, possui grande variância, pois qualquer novo dado adicionado à base, modifica significativamente a posição da margem.

Uma possível saída é a utilização do conceito de *classificador de margem suave*. Um classificador de margem suave é aquele permite que pequenos erros sejam cometidos no momento do treinamento, pois considera que todos pontos que estão dentro da margem como *outliers*.

Esse procedimento é conhecido como *regularização*, que é como uma forma de forçar a simplificação do modelo para garantir que a variância do modelo não seja tão alta. Assim, permite-se um aumento do viés do modelo, em troca da diminuição da variância.

Esse classificador de **margem suave** também é conhecido como **classificador suportado por vetores**, onde esses vetores são os hiperplanos que limitam a margem.

Uma última informação importante a respeito dos classificadores de margem suave:

> A **fronteira de decisão** de um classificador de margem suave (de vetores de suporte) é **linear** no espaço de features em que o classificador é treinado, ou seja, a hipótese treinada (isto é, a superfície de decisão) será **um hiperplano** de dimensão $D-1$, onde $D$ é a dimensão do espaço de features

In [18]:
import warnings
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split

# ignorar warnings
warnings.filterwarnings('ignore')

In [19]:
# montando drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [20]:
# procurando arquivos no Drive
data = pd.read_csv('/content/drive/MyDrive/Bootcamp_DataScience/Algoritmos de Inteligência Artificial para Classificação/datasets/german_credit.csv')
data.head()

,Creditability,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,...,Duration in Current address,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
0,1,1,18,4,2,1049,1,2,4,2,...,4,2,21,3,1,1,3,1,1,1
1,1,1,9,4,0,2799,1,3,2,3,...,2,1,36,3,1,2,3,2,1,1
2,1,2,12,2,9,841,2,4,2,2,...,4,1,23,3,1,1,2,1,1,1
3,1,1,12,4,0,2122,1,3,3,3,...,2,1,39,3,1,2,2,2,1,2
4,1,1,12,4,0,2171,1,3,4,3,...,4,2,38,1,2,2,2,1,1,2


In [21]:
# modificando os nomes das colunas
data.columns = data.columns.str.lower().str.replace(' ', '_')

In [22]:
# separando x e y - vamos usar todos as colunas
x = data.drop(['creditability'], axis=1)
y = data[['creditability']]

In [23]:
# separando treino e teste - com estratificação
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y)

In [24]:
# criando um pipeline de modelagem
pipe_svc = Pipeline([
    ('scaler', MinMaxScaler()),
    ('svc', SVC(random_state=2))
])

pipe_svc.fit(x_train, y_train)

Pipeline(steps=[('scaler', MinMaxScaler()), ('svc', SVC(random_state=2))])

In [25]:
# realizando novas previsões
yhat_train = pipe_svc.predict(x_train)
yhat_test = pipe_svc.predict(x_test)

In [26]:
# análise do desempenho
print('Desempenho - Base de Treino')
print(classification_report(y_train, yhat_train))

Desempenho - Base de Treino
              precision    recall  f1-score   support

           0       0.81      0.51      0.63       210
           1       0.82      0.95      0.88       490

    accuracy                           0.82       700
   macro avg       0.81      0.73      0.75       700
weighted avg       0.82      0.82      0.80       700



In [27]:
# análise do desempenho
print('Desempenho - Base de Treino')
print(classification_report(y_test, yhat_test))

Desempenho - Base de Treino
              precision    recall  f1-score   support

           0       0.70      0.47      0.56        90
           1       0.80      0.91      0.85       210

    accuracy                           0.78       300
   macro avg       0.75      0.69      0.71       300
weighted avg       0.77      0.78      0.77       300



Mas o que acontece se o dataset não for linearmente separável? Nesse caso, não conseguiremos usar o classificador de margem suave. Pelo menos, **não no espaço original dos atributos.**

Mas e se conseguirmos aplicar alguma função que leve os dados para outro espaço de coordenadas de forma que, nesse novo espaço, os dados sejam linearmente separáveis? Então, nesse novo espaço de coordenadas, poderemos aplicar o SVM.

Essa combinação de uma nova função com a aplicação sequencial do SVM é conhecido como mapeamento de atributos (ou *feature map*) e constitui a origem dos modelos chamados *kernel SVM*.

## **Kernel SVM**

A função de kernel é, em resumo, a função não linear que vai levar os dados do espaço original (não linearmente separável) para o novo espaço, de tal forma que os dados sejam linearmente separáveis e, portanto, permitam a aplicação do classificador de margem suave.

As funções de kernel mais comum são:

- linear = SVM comum
- polinomial
- Radial Basis Function, também conhecido como "Kernel Gaussiano"
- Sigmoidal
- Cossenoidal

Essencialmente, a escolha da função de kernel se resume a uma busca de hiperparâmetros.

## **Outros Hiperparâmetros do Modelo SVM**

Outros hiperparâmetros importantes são:

- $C$ - é um parâmetro de regularização, relacionado com a "suavidade" da margem. Ele controla o tradeoff entre a complexidade da fronteira de decisão, e erros de classificação que são permitidos. Quanto **menor** o C, mais suave será a fronteira de decisão, pois mais erros de classificação são permitidos (isto é, a margem fica **mais larga**); quanto **maior** C, a tolerância a erros de classificação é menor (e a margem fica menos suave, mais complexa);

<img src="https://learnopencv.com/wp-content/uploads/2018/07/svm-parameter-c-example.png" width=500>

- `gamma`: define a influência que cada ponto tem na fronteira de decisão. É a "abertura" do kernel: quanto **maior** o gamma, a influência é de mais curto alcance, e vice-versa;

<img src="https://sgao323.gitbooks.io/artificial-intelligence-projects/content/assets/svm_gamma.png" width=400>

- `degree`: aplicado apenas ao kernel polinomial, pois representa o grau polinomial utilizado.

In [28]:
# criando um pipeline de modelagem
pipe_ksvc = Pipeline([
    ('scaler', MinMaxScaler()),
    ('svc', SVC(random_state=2))
])

# configurando um espaço de busca
params_grid = {
    'svc__kernel': ['rbf', 'sigmoid', 'cosine'],
    'svc__C': np.random.uniform(0.01, 10, 100),
    'svc__gamma': np.random.uniform(0.01, 10, 100),
    # 'svc__degree': [1, 2, 3]
}

# configurando um amostrador de k folhas de forma estratificada
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)

# configurando um buscador de hiperparâmetros
random_search = RandomizedSearchCV(
    estimator=pipe_ksvc,
    param_distributions=params_grid,
    n_iter=100,
    scoring='precision_weighted',
    cv=splitter,
    refit=True,
    error_score=0,
    verbose=10
)

# realizando busca
random_search.fit(x_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 1/10; 1/100] START svc__C=4.722507176361753, svc__gamma=2.331492796763891, svc__kernel=sigmoid
[CV 1/10; 1/100] END svc__C=4.722507176361753, svc__gamma=2.331492796763891, svc__kernel=sigmoid;, score=0.490 total time=   0.0s
[CV 2/10; 1/100] START svc__C=4.722507176361753, svc__gamma=2.331492796763891, svc__kernel=sigmoid
[CV 2/10; 1/100] END svc__C=4.722507176361753, svc__gamma=2.331492796763891, svc__kernel=sigmoid;, score=0.490 total time=   0.0s
[CV 3/10; 1/100] START svc__C=4.722507176361753, svc__gamma=2.331492796763891, svc__kernel=sigmoid
[CV 3/10; 1/100] END svc__C=4.722507176361753, svc__gamma=2.331492796763891, svc__kernel=sigmoid;, score=0.490 total time=   0.0s
[CV 4/10; 1/100] START svc__C=4.722507176361753, svc__gamma=2.331492796763891, svc__kernel=sigmoid
[CV 4/10; 1/100] END svc__C=4.722507176361753, svc__gamma=2.331492796763891, svc__kernel=sigmoid;, score=0.490 total time=   0.0s
[CV 5/10; 1/100] ST

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=2, shuffle=True),
                   error_score=0,
                   estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                             ('svc', SVC(random_state=2))]),
                   n_iter=100,
                   param_distributions={'svc__C': array([3.72735299, 4.85281344, 9.37330122, 4.27901233, 4.9477871 ,
       3.98841232, 9.42356759, 2.94839724, 0.61150164, 2.58428869,
       4.95190243, 2.07944...
       9.54633246, 9.12463328, 2.00936164, 4.10236563, 0.30734518,
       8.76739559, 2.67223294, 3.49751626, 3.7821041 , 6.08484245,
       0.53440797, 0.24394077, 9.45449858, 7.05896268, 9.69592117,
       5.09016892, 4.18788119, 8.38585121, 9.78029444, 5.52010127,
       6.49856864, 1.20886439, 2.08337776, 3.14151238, 0.85734885]),
                                        'svc__kernel': ['rbf', 'sigmoid',
                                                        'cosine']},
                   scoring='precision_weighted', verbose=10)

In [29]:
# analisando a melhor combinação de parâmetros
random_search.best_params_

{'svc__kernel': 'rbf',
 'svc__gamma': 0.06715696761736976,
 'svc__C': 5.423782424009382}

In [30]:
# analisando a melhor desempenho médio
random_search.best_score_

0.7528942956736733

In [31]:
# analisando os desempenhos obtidos
precision_cv = []
for i in range(10):
  precision_cv.append(random_search.cv_results_[f'split{i}_test_score'][np.where(random_search.cv_results_['rank_test_score']==1)][0])

precision_cv

[0.743103448275862,
 0.7888257575757576,
 0.7767857142857143,
 0.7428571428571429,
 0.7883333333333334,
 0.7956815114709852,
 0.6856702619414483,
 0.6410714285714286,
 0.7603238866396761,
 0.806290471785384]

In [32]:
# analisando o desempenho final
# realizando novas previsões
yhat_train = random_search.best_estimator_.predict(x_train)
yhat_test = random_search.best_estimator_.predict(x_test)

# análise do desempenho
print('Desempenho - Base de Treino')
print(classification_report(y_train, yhat_train))

print('Desempenho - Base de Teste')
print(classification_report(y_test, yhat_test))

Desempenho - Base de Treino
              precision    recall  f1-score   support

           0       0.74      0.47      0.57       210
           1       0.80      0.93      0.86       490

    accuracy                           0.79       700
   macro avg       0.77      0.70      0.72       700
weighted avg       0.78      0.79      0.77       700

Desempenho - Base de Teste
              precision    recall  f1-score   support

           0       0.70      0.49      0.58        90
           1       0.81      0.91      0.85       210

    accuracy                           0.78       300
   macro avg       0.75      0.70      0.71       300
weighted avg       0.77      0.78      0.77       300



In [33]:
def customer_profit(yreal, ypred):

  # calcula a matriz de confusão
  conf_matrix = confusion_matrix(yreal, ypred)

  # extrai pontuações
  TP = conf_matrix[1, 1]  # Verdadeiros positivos
  FP = conf_matrix[0, 1]  # Falsos positivos
  TN = conf_matrix[0, 0]  # Verdadeiros negativos
  FN = conf_matrix[1, 0]  # Falsos negativos

  # calcula custo
  return ((50 * TP) + (-5 * FN) + (-5 * TN) + (-150 * FP)) / yreal.shape[0]

In [34]:
print('Lucro Médio - Base de Treinamento:', customer_profit(y_train, yhat_train))
print('Lucro Médio - Base de Teste:', customer_profit(y_test, yhat_test))

Lucro Médio - Base de Treinamento: 7.55
Lucro Médio - Base de Teste: 7.783333333333333


In [35]:
# configurando um buscador de hiperparâmetros
random_search = RandomizedSearchCV(
    estimator=pipe_ksvc,
    param_distributions=params_grid,
    n_iter=100,
    scoring=make_scorer(customer_profit, greater_is_better=True),
    cv=splitter,
    refit=True,
    error_score=0,
    verbose=10
)

# realizando busca
random_search.fit(x_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 1/10; 1/100] START svc__C=3.642241791751393, svc__gamma=4.776603828036206, svc__kernel=sigmoid
[CV 1/10; 1/100] END svc__C=3.642241791751393, svc__gamma=4.776603828036206, svc__kernel=sigmoid;, score=-10.000 total time=   0.1s
[CV 2/10; 1/100] START svc__C=3.642241791751393, svc__gamma=4.776603828036206, svc__kernel=sigmoid
[CV 2/10; 1/100] END svc__C=3.642241791751393, svc__gamma=4.776603828036206, svc__kernel=sigmoid;, score=-10.000 total time=   0.0s
[CV 3/10; 1/100] START svc__C=3.642241791751393, svc__gamma=4.776603828036206, svc__kernel=sigmoid
[CV 3/10; 1/100] END svc__C=3.642241791751393, svc__gamma=4.776603828036206, svc__kernel=sigmoid;, score=-10.000 total time=   0.0s
[CV 4/10; 1/100] START svc__C=3.642241791751393, svc__gamma=4.776603828036206, svc__kernel=sigmoid
[CV 4/10; 1/100] END svc__C=3.642241791751393, svc__gamma=4.776603828036206, svc__kernel=sigmoid;, score=-10.000 total time=   0.0s
[CV 5/10; 1

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=2, shuffle=True),
                   error_score=0,
                   estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                             ('svc', SVC(random_state=2))]),
                   n_iter=100,
                   param_distributions={'svc__C': array([3.72735299, 4.85281344, 9.37330122, 4.27901233, 4.9477871 ,
       3.98841232, 9.42356759, 2.94839724, 0.61150164, 2.58428869,
       4.95190243, 2.07944...
       9.54633246, 9.12463328, 2.00936164, 4.10236563, 0.30734518,
       8.76739559, 2.67223294, 3.49751626, 3.7821041 , 6.08484245,
       0.53440797, 0.24394077, 9.45449858, 7.05896268, 9.69592117,
       5.09016892, 4.18788119, 8.38585121, 9.78029444, 5.52010127,
       6.49856864, 1.20886439, 2.08337776, 3.14151238, 0.85734885]),
                                        'svc__kernel': ['rbf', 'sigmoid',
                                                        'cosine']},
                   scoring=make_scorer(customer_profit), verbose=10)

In [36]:
# analisando a melhor combinação de parâmetros
random_search.best_params_

{'svc__kernel': 'rbf',
 'svc__gamma': 0.7031674155366987,
 'svc__C': 7.672949942400363}

In [37]:
# analisando a melhor desempenho médio
random_search.best_score_

5.500000000000001

In [38]:
# analisando o desempenho final
# realizando novas previsões
yhat_train = random_search.best_estimator_.predict(x_train)
yhat_test = random_search.best_estimator_.predict(x_test)

# análise do desempenho
print('Desempenho - Base de Treino')
print(classification_report(y_train, yhat_train))

print('Desempenho - Base de Teste')
print(classification_report(y_test, yhat_test))

Desempenho - Base de Treino
              precision    recall  f1-score   support

           0       0.98      0.95      0.96       210
           1       0.98      0.99      0.98       490

    accuracy                           0.98       700
   macro avg       0.98      0.97      0.97       700
weighted avg       0.98      0.98      0.98       700

Desempenho - Base de Teste
              precision    recall  f1-score   support

           0       0.53      0.51      0.52        90
           1       0.79      0.81      0.80       210

    accuracy                           0.72       300
   macro avg       0.66      0.66      0.66       300
weighted avg       0.72      0.72      0.72       300



In [39]:
print('Lucro Médio - Base de Treinamento:', customer_profit(y_train, yhat_train))
print('Lucro Médio - Base de Teste:', customer_profit(y_test, yhat_test))

Lucro Médio - Base de Treinamento: 30.90714285714286
Lucro Médio - Base de Teste: 4.9
